In [ ]:
import sys

sys.path.insert(0, "/home/esrf/james1997a/Code/ImageD11")

In [ ]:
from ImageD11.grain import grain
from ImageD11.unitcell import unitcell
import numpy as np
from scipy.spatial.transform import Rotation as R

from orix.quaternion import Orientation, Rotation
from orix.quaternion.symmetry import get_point_group
from orix.vector import Miller
from orix.crystal_map import Phase
from orix.plot import IPFColorKeyTSL
from orix.vector.vector3d import Vector3d

from diffpy.structure import Lattice, Structure

from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
hexagonal_ucell_array = np.array([2.94, 2.94, 4.72, 90, 90, 120])
hexagonal_ucell = unitcell(hexagonal_ucell_array, symmetry=194)
B = hexagonal_ucell.B

In [ ]:
# make a reference unit cell too

ref_hexagonal_ucell_array = np.array([2.94, 2.94, 4.72, 90, 90, 120])
ref_hexagonal_ucell = unitcell(ref_hexagonal_ucell_array, symmetry=194)

In [ ]:
# generate 1000 random orientations

Us = R.random(10000).as_matrix()

In [ ]:
UBs = Us @ B

In [ ]:
def plot_inverse_pole_figure(ref_ucell, UBs, axis=np.array([0., 0, 1])):
    # get a meta orientation for all the grains
    meta_orien = ref_ucell.get_orix_orien(UBs)

    try:
        from orix.vector.vector3d import Vector3d
    except ImportError:
        raise ImportError("Missing diffpy and/or orix, can't compute orix phase!")

    ipf_direction = Vector3d(axis)

    # get the RGB colours
    rgb = ref_ucell.get_ipf_colour_from_orix_orien(meta_orien, axis=ipf_direction)

    # scatter the meta orientation using the colours
    meta_orien.scatter("ipf", c=rgb, direction=ipf_direction)

In [ ]:
plot_inverse_pole_figure(ref_hexagonal_ucell, UBs)

In [ ]:
def plot_all_ipfs(ref_ucell, UBs):
    plot_inverse_pole_figure(ref_ucell, UBs, axis=np.array([1., 0, 0]))
    plot_inverse_pole_figure(ref_ucell, UBs, axis=np.array([0., 1, 0]))
    plot_inverse_pole_figure(ref_ucell, UBs, axis=np.array([0., 0, 1]))

In [ ]:
plot_all_ipfs(hexagonal_ucell, UBs)

In [ ]:
# generate 10 orientations clustered near identity
# they'll all be rotated around x and y such that c* doesn't point exactly || z

Us = R.from_euler(seq='xy', angles=(np.random.random((10, 2))-0.5)*25, degrees=True).as_matrix()

UBs = Us @ B

plot_inverse_pole_figure(hexagonal_ucell, UBs)

In [ ]:
def pf_plot_UBs(ref_ucell, UBs, uvw=np.array([1., 0., 0.])):
    # make a combined orientation from them (makes plot much faster)
    meta_orien = ref_ucell.get_orix_orien(UBs)
    
    # make Miller object from uvw
    m1 = Miller(uvw=uvw, phase=ref_ucell.orix_phase).symmetrise(unique=True)
    
    # get outer product of all orientations with the crystal direction we're interested in
    uvw_all = (~meta_orien).outer(m1)
    
    uvw_all.scatter(hemisphere="both", axes_labels=["X", "Y"])

In [ ]:
# let's test a triclinic too:

# Labradorite
# ICSD 100232
# 8.1736 12.8736 7.1022 93.462 116.054 90.475
# space group 2
triclinic_ucell_array = np.array([8.1736, 12.8736, 7.1022, 93.462, 116.054, 90.475])
triclinic_ucell = unitcell(triclinic_ucell_array, symmetry=2)

B = triclinic_ucell.B

# establish some orientations that are all close to a* || x
# we do this by rotating around y and z slightly

Us = R.from_euler(seq='yz', angles=(np.random.random((10000, 2))-0.5)*15, degrees=True).as_matrix()

UBs = Us @ B

# U is identity
# a* should be along x_lab

uvw = Miller(hkl=(1,0,0),phase=triclinic_ucell.orix_phase).uvw

pf_plot_UBs(triclinic_ucell, UBs, uvw)